In [11]:
!pip install ultralytics

from ultralytics import YOLO
import yaml, copy, os


In [12]:
# مسار وزن النموذج المدرّب على Retail
teacher_weights = "/kaggle/input/yolo-8/transformers/default/1/best (1).pt"

teacher = YOLO(teacher_weights)
teacher_model = teacher.model

print("Teacher model type:", type(teacher_model))
print("Original YAML nc, depth, width:",
      teacher_model.yaml['nc'],
      teacher_model.yaml['depth_multiple'],
      teacher_model.yaml['width_multiple'])


Teacher model type: <class 'ultralytics.nn.tasks.DetectionModel'>
Original YAML nc, depth, width: 1 0.67 0.75


In [13]:
orig_yaml = teacher_model.yaml
stage_yaml = copy.deepcopy(orig_yaml)

stage_yaml


{'nc': 1,
 'depth_multiple': 0.67,
 'width_multiple': 0.75,
 'backbone': [[-1, 1, 'Conv', [64, 3, 2]],
  [-1, 1, 'Conv', [128, 3, 2]],
  [-1, 3, 'C2f', [128, True]],
  [-1, 1, 'Conv', [256, 3, 2]],
  [-1, 6, 'C2f', [256, True]],
  [-1, 1, 'Conv', [512, 3, 2]],
  [-1, 6, 'C2f', [512, True]],
  [-1, 1, 'Conv', [768, 3, 2]],
  [-1, 3, 'C2f', [768, True]],
  [-1, 1, 'SPPF', [768, 5]]],
 'head': [[-1, 1, 'nn.Upsample', ['None', 2, 'nearest']],
  [[-1, 6], 1, 'Concat', [1]],
  [-1, 3, 'C2f', [512]],
  [-1, 1, 'nn.Upsample', ['None', 2, 'nearest']],
  [[-1, 4], 1, 'Concat', [1]],
  [-1, 3, 'C2f', [256]],
  [-1, 1, 'Conv', [256, 3, 2]],
  [[-1, 12], 1, 'Concat', [1]],
  [-1, 3, 'C2f', [512]],
  [-1, 1, 'Conv', [512, 3, 2]],
  [[-1, 9], 1, 'Concat', [1]],
  [-1, 3, 'C2f', [768]],
  [[15, 18, 21], 1, 'Detect', ['nc']]],
 'ch': 3}

✅ Cell 4 — تطبيق “Pruning موجّه” على الـ Backbone والـ Neck

فكرة مبسطة:

YOLOv8m backbone في YAML تقريباً (كما تعرف):

backbone:
  [[-1, 1, 'Conv', [64, 3, 2]],   # stage1
   [-1, 1, 'Conv', [128, 3, 2]],  # stage2
   [-1, 3, 'C2f', [128, True]],   # C2f s2
   [-1, 1, 'Conv', [256, 3, 2]],  # stage3
   [-1, 6, 'C2f', [256, True]],   # C2f s3
   [-1, 1, 'Conv', [512, 3, 2]],  # stage4
   [-1, 6, 'C2f', [512, True]],   # C2f s4
   [-1, 1, 'Conv', [768, 3, 2]],  # stage5
   [-1, 3, 'C2f', [768, True]],
   [-1, 1, 'SPPF', [768, 5]]]


نريد أن نعمل:

Prune موجّه على:

C2f في stage2 (128 → 96 مثلاً)

C2f في stage3 (256 → 192)

C2f في stage4 (512 → 384)

وبعض C2f في الـ head (512 → 384, 256 → 192, 768 → 576)

الكود التالي يفعل ذلك تلقائياً:

In [14]:
def scale_channels(ch, ratio, make_divisible=8):
    new_ch = int(ch * ratio)
    new_ch = max(make_divisible, (new_ch // make_divisible) * make_divisible)
    return new_ch

prune_ratio_backbone = 0.75  # 25% تقليل
prune_ratio_head     = 0.75

# 1) تعديل الـ backbone channels
for i, block in enumerate(stage_yaml['backbone']):
    from_idx, n, m, args = block
    if m in ['Conv', 'C2f', 'SPPF']:
        ch = args[0]
        # نركز على المراحل المتوسطة: 128, 256, 512
        if ch in [128, 256, 512]:
            new_ch = scale_channels(ch, prune_ratio_backbone)
            print(f"Backbone block {i} : {m} {ch} -> {new_ch}")
            args[0] = new_ch
            stage_yaml['backbone'][i][3] = args

# 2) تعديل الـ head channels
for i, block in enumerate(stage_yaml['head']):
    from_idx, n, m, args = block
    if m == 'C2f':
        ch = args[0]
        if ch in [256, 512, 768]:
            new_ch = scale_channels(ch, prune_ratio_head)
            print(f"Head block {i} : {m} {ch} -> {new_ch}")
            args[0] = new_ch
            stage_yaml['head'][i][3] = args

print("Pruning-oriented channel scaling applied.")


Backbone block 1 : Conv 128 -> 96
Backbone block 2 : C2f 128 -> 96
Backbone block 3 : Conv 256 -> 192
Backbone block 4 : C2f 256 -> 192
Backbone block 5 : Conv 512 -> 384
Backbone block 6 : C2f 512 -> 384
Head block 2 : C2f 512 -> 384
Head block 5 : C2f 256 -> 192
Head block 8 : C2f 512 -> 384
Head block 11 : C2f 768 -> 576
Pruning-oriented channel scaling applied.


In [15]:
stage_yaml['nc'] = orig_yaml['nc']  # عدد الكلاسات يبقى كما هو

pruned_yaml_path = "/kaggle/working/yolov8m_stagepruned.yaml"
with open(pruned_yaml_path, "w") as f:
    yaml.dump(stage_yaml, f)

print("Saved stage-pruned YAML to:", pruned_yaml_path)


Saved stage-pruned YAML to: /kaggle/working/yolov8m_stagepruned.yaml


In [16]:
from ultralytics import YOLO

student = YOLO(pruned_yaml_path)
student.info()  # يعرض layers, params, GFLOPs

print("Stage-pruned model created successfully.")


YOLOv8m_stagepruned summary: 169 layers, 17,687,779 parameters, 17,687,763 gradients, 48.5 GFLOPs
Stage-pruned model created successfully.


In [17]:
baseline = YOLO("yolov8m.pt")  # أو استخدم teacher.model.yaml مباشرة
print("\nBaseline model:")
baseline.info()

print("\nStage-pruned model:")
student.info()



Baseline model:
YOLOv8m summary: 169 layers, 25,902,640 parameters, 0 gradients, 79.3 GFLOPs

Stage-pruned model:
YOLOv8m_stagepruned summary: 169 layers, 17,687,779 parameters, 17,687,763 gradients, 48.5 GFLOPs


(169, 17687779, 17687763, 48.467366399999996)